In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from plotly import graph_objects as go

# E-Commerce Conversion Funnel

In [4]:
import os
import re

In [8]:
def extract_file_name(path):
    name_pattern = r"(.*)_page_table\.csv$"
    return re.findall(name_pattern, path)[0]

In [11]:
name_pattern = r"(.*)\_page\_table\.csv$"
re.findall(name_pattern, "payment_confirmation_table.csv")

[]

In [9]:
extract_file_name("payment_confirmation_table.csv")

IndexError: list index out of range

In [3]:
files = {}
for file in os.listdir("data/"):
    
    files[] = 

['search_page_table.csv',
 'payment_page_table.csv',
 'home_page_table.csv',
 'user_table.csv',
 'payment_confirmation_table.csv']

In [12]:
home_page = pd.read_csv("data/home_page_table.csv")
search_page = pd.read_csv("data/search_page_table.csv")
payment_page = pd.read_csv("data/payment_page_table.csv")
payment_confirmation = pd.read_csv("data/payment_confirmation_table.csv")
user = pd.read_csv("data/user_table.csv")

In [26]:
home_page.head()

,user_id,page
0,313593,home_page
1,468315,home_page
2,264005,home_page
3,290784,home_page
4,639104,home_page


In [15]:
user.head()

,user_id,date,device,sex
0,450007,2015-02-28,Desktop,Female
1,756838,2015-01-13,Desktop,Male
2,568983,2015-04-09,Desktop,Male
3,190794,2015-02-18,Desktop,Female
4,537909,2015-01-15,Desktop,Male


In [17]:
user.dtypes

user_id     int64
date       object
device     object
sex        object
dtype: object

In [19]:
user["date"] = pd.to_datetime(user["date"])

## Data prep

For modelling a funnel, we need a dataset that looks like:

| user_id | date | device | sex | home_page | search_page | payment_page | payment_confirmation |
|---------|------|--------|-----|-----------|-------------|--------------|----------------------|
| 13661 | ... | ... | ... | 1 | 1 | 0 | 0 |


In [32]:
df = (user
      .merge(home_page, how="inner", on="user_id")
      .rename(columns={"page": "home_page"})
      .merge(search_page, how="left", on="user_id")
      .rename(columns={"page": "search_page"})
      .merge(payment_page, how="left", on="user_id")
      .rename(columns={"page": "payment_page"})
      .merge(payment_confirmation, how="left", on="user_id")
      .rename(columns={"page": "payment_confirmation"})).fillna(0)
 

In [33]:
df

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,home_page,0,0,0
1,756838,2015-01-13,Desktop,Male,home_page,0,0,0
2,568983,2015-04-09,Desktop,Male,home_page,search_page,0,0
3,190794,2015-02-18,Desktop,Female,home_page,search_page,0,0
4,537909,2015-01-15,Desktop,Male,home_page,0,0,0
...,...,...,...,...,...,...,...,...
90395,307667,2015-03-30,Desktop,Female,home_page,0,0,0
90396,642989,2015-02-08,Desktop,Female,home_page,search_page,0,0
90397,659645,2015-04-13,Desktop,Male,home_page,search_page,0,0
90398,359779,2015-03-23,Desktop,Male,home_page,0,0,0


In [36]:
df[["home_page", "search_page", "payment_page", "payment_confirmation"]] = np.where(df[["home_page", "search_page", "payment_page", "payment_confirmation"]] == 0, 0, 1)

In [38]:
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,1,0,0,0
1,756838,2015-01-13,Desktop,Male,1,0,0,0
2,568983,2015-04-09,Desktop,Male,1,1,0,0
3,190794,2015-02-18,Desktop,Female,1,1,0,0
4,537909,2015-01-15,Desktop,Male,1,0,0,0
